# Descargar Tweet con Tweepy

In [ ]:
#Importamos librerias
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import tweepy
import json
import pandas as pd
import csv
import re
import sys
from textblob import TextBlob
import string
import preprocessor as p
import os
import time


## Llamada a la API y Descarga de Tweets

In [ ]:
# Credenciasles Twitter
# Se obtienen desde tu cuentas de  twitter developer.
consumer_key ='Rellenar con tus credenciales'
consumer_secret ='Rellenar con tus credenciales'
access_key ='Rellenar con tus credenciales'
access_secret ='Rellenar con tus credenciales'

# Pasa tus credenciales de  twitter a tweepy via OAuthHandler.
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth, wait_on_rate_limit = True)

In [ ]:
#Función para descargarse los tweet.

def text_query_to_csv(text_query,count,data_since):
    try:
        all_tweets = []
        
        # Creacción de  queryusando los parametos que necesitamos
        print("Descargando Tweets")
        tweets = tweepy.Cursor(api.search,q=text_query,lang = 'es', locale ='UTF-8',since= data_since, tweet_mode='extended').items(count)
        all_tweets = all_tweets + list(tweets)
        print("Descargados Tweets")
        #Finalmente no necesira time.sleep,lo hace automaticamente poniendo un parametro en la función API.
        #print("Esperando 15 min")
        # esperar 15 min api limit
        #time.sleep(60*15)
            
        # Iteramos para sacar la información que queremos de cada tweet.
        tweets_list = [[tweet.created_at, tweet.id,tweet.user.created_at,tweet.user.screen_name,tweet.user.description,tweet.user.location,
                        tweet.user.friends_count,tweet.user.followers_count,tweet.user.statuses_count,tweet.retweet_count,
                        tweet.full_text,tweet.entities['hashtags']] for tweet in all_tweets]

        # Creaamos DataFrame con la lista de tweets
        tweets_df = pd.DataFrame(tweets_list,columns=['Datetime','Tweet Id','Creacion','Username','Description','Location',
                                                      'Following','Followers','Total Tweets','RT','Text','Hashtags'])

        #Convertimos Df en csv
        tweets_df.to_csv('{}-tweets.csv'.format('#LaIslaDeLasTentaciones5'), sep=',', index = False)

    except BaseException as e:
        print('failed on_status,',str(e))
        #time.sleep(3)

In [ ]:
# Valores con los que vamos a realizar la busqueda.

#Hashtag que vamos a buscar y filtros que aplicamos en la busqueda. 
text_query = '(#LaIslaDeLasTentaciones5) -filter:links -filter:replies -filter:retweets'
#Numero de tweets.
count = 200000
#Fecha desde que empezamos a buscar
data_since ='2021-02-11'


# Llamamos a la función 
text_query_to_csv(text_query,count,data_since)

## Limpieza Tweets

In [ ]:
#Importamos DF creado.
df =pd.read_excel('#LaIslaDeLasTentaciones5-tweets.xlsx')

In [ ]:
df.info()

In [ ]:
df.tail()

### Limpieza general 

In [ ]:
# Función para realizar limpieza general.

def cleanUpTweet(txt):
    # Remove mentions
    txt = re.sub(r'@[A-Za-z0-9_]+', '', txt)
    # Remove hashtags
    txt = re.sub(r'#', '', txt)
    # Remove retweets:
    txt = re.sub(r'RT : ', '', txt)
    # Remove urls
    txt = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', ' ', txt)
    #Remove line break
    txt = re.sub(r'\n', ' ', txt )
    #Remove-
    txt = re.sub(r'-', ' ', txt )
    return txt


In [ ]:
#Aplicamos función

df['Text'] = df['Text'].apply(cleanUpTweet)


In [ ]:
df.tail(5)

### Limpieza abreviaturas, reemplazar palabras y pasar a minuscula

In [ ]:
#Abreviaturas 
abbrevs = {'d': 'de',
               'x': 'por',
               'xa': 'para',
               'as': 'has',
               'q': 'que',
               'ke': 'que',
               'k': 'que',
               'dl': 'del',
               'xq': 'porqué',
               'dr': 'doctor',
               'dra': 'doctora',
               'sr': 'señor',
               'sra': 'señora',
               'm': 'me'}
elementos = abbrevs.items()

In [ ]:
#Función para remplazar abreviaturas, Hashtag y transformar texto en minuscula

re.sub(r"\bis\b", "was", "This is the best island!")
lista_abreviaturas =[]
dfMo = df.copy()
for a, b in elementos:
    #lista_abreviaturas = list(map(lambda x : x.replace(x,i), df['Text']))
    a = " " +a + " "
    b = " " +b+ " "
    
    #dfMo['Text'] = dfMo['Text'].apply(lambda x :re.sub(a, b, x))
    dfMo['Text'] = dfMo['Text'].str.replace(a,b,regex =True)

#Hashtag    

dfMo['Text'] = dfMo['Text'].apply(lambda x : x.replace('LaIslaDeLasTentaciones5','La Isla'))

#Minusculas

dfMo['Text']  = dfMo['Text'].apply(lambda x :x.lower())

print(dfMo['Text'])

    
    

### Limpieza emojis

In [ ]:
#Función para eliminar emojis
def remove_emoji(string):
    emoji_pattern = re.compile("["
                          u"\U0001F600-\U0001F64F"  # emoticonos
                               u"\U0001F300-\U0001F5FF"  # simbolos y pictogramas
                               u"\U0001F680-\U0001F6FF"  # Trasporte y s
                               u"\U0001F1E0-\U0001F1FF"  # Banderas (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  
                               u"\u3030"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)



In [ ]:
#Aplicamos función

dfMo['Text'] = dfMo['Text'].apply(remove_emoji)

In [ ]:
#Guardamos DF con los tweets limpios

dfMo.to_csv('LaIslaDeLastenataciones5-tweets-limpio-completo.csv')

In [ ]:
dfMo.to_excel('LaIslaDeLastenataciones5-tweets-limpio-completo.xlsx')